In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

z = pd.read_csv(r"C:\Users\gromo\OneDrive\Documents\transactions_diy.csv")

# загружаем датасет

In [2]:
# смотрим датасет

z

,TrDte,BCode,ClientID,Item,ItemGroup,Quantity,Amount
0,01.09.2017,code000000001,client13166,sku8444,Скобяные изделия,1,29
1,01.09.2017,code000000001,client13166,sku12545,Оборудование для сада и дачи,1,329
2,01.09.2017,code000000001,client13166,sku3391,Инструменты,1,169
3,01.09.2017,code000000001,client13166,sku20444,Инструменты,2,578
4,01.09.2017,code000000002,client1239,sku29959,Скобяные изделия,1,329
...,...,...,...,...,...,...,...
1008683,31.10.2019,code000290228,client25258,sku5837,Скобяные изделия,1,89
1008684,31.10.2019,code000290228,client25258,sku26161,Стойматериалы,2,598
1008685,31.10.2019,code000290228,client25258,sku20658,Сантехника,1,159
1008686,31.10.2019,code000290229,client22472,sku16069,Столярные изделия,1,85


In [3]:
# смотрим какие типы данных

z.dtypes

TrDte        object
BCode        object
ClientID     object
Item         object
ItemGroup    object
Quantity      int64
Amount        int64
dtype: object

In [4]:
# замечаем, что TrDte - object




In [5]:
# Для первого задания мы создадим отдельный датафрейм с сортировкой Item по сумме Amount
# Полученный результат отсортируем по столбцу Amount по убыванию
# Выведем строку с индексом 9 (начало нумирации - 0)

z1 = z.groupby(['Item']).sum(['Amount'])
z1 = z1.sort_values(by=['Amount'], ascending=[0])
z1.iloc[9]


Quantity       8384
Amount      1645042
Name: sku27791, dtype: int64

In [6]:
# Второе задание

z['Item'].unique().shape[0]

30566

In [7]:
# Для третьего задания мы создадим новый датафрейм, но с фильтрацией по столбцу TrDte
# Условие фильтрации TrDte > 01.11.2019
# Также создаем датафрейм по отсоритрованным данным с сортировкой Item по сумме Amount
# Полученный результат отсортируем по столбцу Amount по убыванию
# Выведем строку с индексом 0 (начало нумирации - 0)

z2 = z[(z['TrDte'] > '01.11.2019')]
z2 = z2.groupby(['Item']).sum(['Amount'])
z2 = z2.sort_values(by=['Amount'], ascending=[0])
z2.iloc[0]

Quantity      18374
Amount      5221678
Name: sku26161, dtype: int64

In [8]:
# Для четвертого задания сначала создаем столбец для года/месяца
# В этих столбцах мы сначала выводим содержание столбца TrDte как datetime
# Столбцы преобразуем в месяц/год
# Группируем по году и месяцу уникальные значения в датафрейме
# Выводим полученный датафрейм

z['Month'] = z['TrDte']
z['Month'] = pd.to_datetime(z['Month'], dayfirst=True)
z['Month'] = z['Month'].dt.month

z['Year'] = z['TrDte']
z['Year'] = pd.to_datetime(z['Year'], dayfirst=True)
z['Year'] = z['Year'].dt.year

z3 = z[['Month', 'Year', 'BCode']]
z3 = z3.groupby(by=['Year', 'Month']).nunique()
z3 = z3.reset_index()

z3

,Year,Month,BCode
0,2017,9,2708
1,2017,10,4095
2,2017,11,4622
3,2017,12,4432
4,2018,1,5108
5,2018,2,5913
6,2018,3,7418
7,2018,4,8099
8,2018,5,9315
9,2018,6,7315


In [9]:
# Фильтруем df по году/месяцу
# Переводим Amount во флоат
# считаем средние и стандартное отклонение по всему периоду
# считаем вариацию
# по логике расствляем классы по ABC
# далее там же считаем окно сумма Amount по продукту, после фильтруем по продукту (это необязательно)
# убираем дубли пр названию продукта, нам больше ничего не нужно
# Сортируем по убыванию суммы Amount
# Считаем все продажи по периоду и проценту суммы Amount продукта от общей
# СЧитаем кумулянту по сумме проценту суммы Amount продукта
# По кумулянте выставляем классы ABC, фильтруем, выводим длину

z4 = z[['Item', 'Quantity', 'Amount', 'Year', 'Month']]
z4 = z4[(z4['Year'] == 2019) & (z4['Month'] >=5)]
z4['Amount'].astype('float64')
z4['mean'] = z4.groupby(['Item'])['Quantity'].transform('mean')
z4['std'] = z4.groupby(['Item'])['Quantity'].transform('std')
z4['var'] = z4['std'] / z4['mean']
z4['XYZ'] = ['Z' if x >= 0.25 else 'Y' if x >= 0.1 else 'X' for x in z4['var']]

z4 = z4[['Item', 'Amount', 'XYZ', 'Year', 'Month', 'Quantity', 'std', 'var', 'mean']]

z5 = z4
z5['group_amount'] = z5.groupby(['Item'])['Amount'].transform('sum')
z5 = z5.sort_values(by=['Item'], ascending=[0])

z5 = z5.drop_duplicates(subset=['Item'])
Z5 = z5.sort_values(by=['Amount'], ascending=[0])

z5['All_sales'] = z5['group_amount'].sum()

z5['Percent_sales'] = z5['group_amount'] / z5['All_sales'] * 100

z5 = z5.sort_values(by=['Percent_sales'], ascending=[0])
z5['Cum_percent'] = z5['Percent_sales'].cumsum()
z5['ABC'] = ['C' if x >= 95 else 'B' if x >= 80 else 'A' for x in z5['Cum_percent']]

z5_1 = z5[['Item', 'ABC', 'XYZ']]
z5 = z5[(z5['XYZ'] == 'X') & (z5['ABC'] == 'A')]
z5 = z5.sort_values(by=['Percent_sales'], ascending=[0])
z5.shape[0]

1302

In [10]:
# В шестом задании

z6 = z[['Item', 'Quantity', 'Amount', 'Year', 'Month', 'TrDte']]
# z6 = z6[(z6['TrDte'] < '01.05.2019')]
z6['Amount'].astype('float64')
z6['mean1'] = z6.groupby(['Item'])['Quantity'].transform('mean')
z6['std1'] = z6.groupby(['Item'])['Quantity'].transform('std')
z6['var1'] = z6['std1'] / z6['mean1']
z6['XYZ1'] = ['Z' if x >= 0.25 else 'Y' if x >= 0.1 else 'X' for x in z6['var1']]

z7 = z6
z7['group_amount1'] = z7.groupby(['Item'])['Amount'].transform('sum')
z7 = z7.sort_values(by=['Item'], ascending=[0])

z7 = z7.drop_duplicates(subset=['Item'])

z7['All_sales1'] = z7['group_amount1'].sum()

z7['Percent_sales1'] = z7['group_amount1'] / z7['All_sales1'] * 100

z7 = z7.sort_values(by=['Percent_sales1'], ascending=[0])
z7['Cum_percent1'] = z7['Percent_sales1'].cumsum()
z7['ABC1'] = ['C' if x >= 95 else 'B' if x >= 80 else 'A' for x in z7['Cum_percent1']]

z8 = pd.merge(z7, z5_1, how='left', on=['Item'])
z8 = z8[['Item', 'ABC', 'XYZ', 'ABC1', 'XYZ1']]
z8['ABC_XYZ'] = z8['ABC'] + z8['XYZ']
z8['ABC_XYZ1'] = z8['ABC1'] + z8['XYZ1']


z8.dropna()
z8 = z8.loc[(z8['ABC_XYZ'] != z8['ABC_XYZ1'])]
z8.dropna()


C:\Users\gromo\AppData\Local\Temp\ipykernel_6924\1668328760.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z6['mean1'] = z6.groupby(['Item'])['Quantity'].transform('mean')
C:\Users\gromo\AppData\Local\Temp\ipykernel_6924\1668328760.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z6['std1'] = z6.groupby(['Item'])['Quantity'].transform('std')


,Item,ABC,XYZ,ABC1,XYZ1,ABC_XYZ,ABC_XYZ1
7,sku9471,C,X,A,Z,CX,AZ
41,sku1277,A,Y,A,Z,AY,AZ
75,sku16598,A,X,A,Y,AX,AY
90,sku12097,A,X,A,Z,AX,AZ
92,sku4798,A,Z,A,Y,AZ,AY
...,...,...,...,...,...,...,...
30018,sku24827,C,X,C,Z,CX,CZ
30070,sku13907,C,X,C,Z,CX,CZ
30123,sku13531,C,X,C,Z,CX,CZ
30148,sku15024,C,X,C,Z,CX,CZ
